# LAN Sweep Function
## IP Conversion Functions

In [ ]:
def IP_str_to_int(IP):
    # Converts an IP address stored as a string, i.e. '192.168.0.0', to an integer
    first_dot = IP.find('.')
    first_octet = int(IP[0:first_dot])
    second_dot = IP.find('.', first_dot + 1)
    second_octet = int(IP[first_dot + 1:second_dot])
    third_dot = IP.find('.', second_dot + 1)
    third_octet = int(IP[second_dot + 1:third_dot])
    fourth_octet = int(IP[third_dot + 1:len(IP)])

    IP_int = (first_octet * 256**3) + (second_octet * 256**2) + (third_octet * 256) + fourth_octet
    
    return IP_int

def IP_int_to_str(IP):
    # Converts an IP address stored as an integer to a string, i.e. '192.168.0.0'
    fourth_octet = IP % 256
    IP = IP // 256
    third_octet = IP % 256
    IP = IP // 256
    second_octet = IP % 256
    IP = IP // 256
    first_octet = IP % 256
    
    IP_str = str(first_octet) + '.' + str(second_octet) + '.' + str(third_octet) + '.' + str(fourth_octet)
    
    return IP_str

def IP_str_to_bin(IP):
    # Returns a string of the IP address in binary format
    IP_int = IP_str_to_int(IP)
    IP_bin = '{0:b}'.format(IP_int)
    
    while len(IP_bin) < 32:
        IP_bin = '0' + IP_bin

    return IP_bin

def IP_bin_to_str(IP):
    # Converts an IP address stored as a string of 0's and 1's to a string
    IP_int = int(IP, 2)
    IP_str = IP_int_to_str(IP_int)
    return IP_str

def subnet_mask_to_CIDR(subnet_mask):
    # Converts a subnet mask stored as a string to the equivalent CIDR notation
    subnet_mask_bin = IP_str_to_bin(subnet_mask)
    bits = subnet_mask_bin.count('1')
    CIDR = '/' + str(bits)
    
    return CIDR

def CIDR_to_subnet_mask(CIDR):
    # Converts a CIDR notation subnet to a subnet mask
    if '/' in CIDR:
        CIDR_int = CIDR[1:]
    else:
        CIDR_int = CIDR
        
    IP_bin = ''
    
    while len(IP_bin) < int(CIDR_int):
        IP_bin += '1'
    
    while len(IP_bin) < 32:
        IP_bin += '0'
        
    subnet_mask = IP_bin_to_str(IP_bin)
        
    return subnet_mask

## Get Network Value Functions

In [ ]:
import netifaces
import socket

def get_host_IP_address():
    host_IP = socket.gethostbyname(socket.gethostname())
    return host_IP

def get_hostname(IP = get_host_IP_address()):
    try:
        hostname = socket.gethostbyaddr(IP)[0]
    except:
        hostname = ''
    return hostname    

def get_default_gateway():
    # Returns the default gateway
    gateways = netifaces.gateways()
    default_gateway = gateways['default'][netifaces.AF_INET][0]
    return default_gateway

def get_broadcast_address():
    # Returns the default broadcast address
    gateways = netifaces.gateways()
    default_interface = gateways['default'][netifaces.AF_INET][1]
    addrs = netifaces.ifaddresses(default_interface)
    broadcast_address = addrs[netifaces.AF_INET][0]['broadcast']
    return broadcast_address

def get_subnet_mask():
    # Returns the default subnet mask
    gateways = netifaces.gateways()
    default_interface = gateways['default'][netifaces.AF_INET][1]
    addrs = netifaces.ifaddresses(default_interface)
    subnet_mask = addrs[netifaces.AF_INET][0]['netmask']
    return subnet_mask

def get_network_address():
    # Returns the default network address
    host_IP = get_host_IP_address()
    subnet_mask = get_subnet_mask()
    
    host_IP_bin = IP_str_to_bin(host_IP)
    subnet_mask_bin = IP_str_to_bin(subnet_mask)
    
    network_address_bin = ''
    for IP_bin, subnet_bin in zip(host_IP_bin, subnet_mask_bin):
        if subnet_bin == '1':
            network_address_bin += IP_bin
        elif subnet_bin == '0':
            network_address_bin += '0'
        else:
            raise ValueError("Binary IP addresses should only contain 0's and 1's")
            
    network_address = IP_bin_to_str(network_address_bin)
        
    return network_address

def get_LAN():
    # Returns the LAN in CIDR notation
    network_address = get_network_address()
    subnet_mask = get_subnet_mask()
    
    CIDR = subnet_mask_to_CIDR(subnet_mask)
    
    LAN = network_address + CIDR
    
    return LAN

## Other Network Functions

In [ ]:
import pandas as pd

def network_IP_list(network_address, subnet_mask):
    # Returns a list of the IP addresses within an IP network
    subnet_mask_bin = IP_str_to_bin(subnet_mask)
    zeros = subnet_mask_bin.count('0')
    size = 2**zeros
    
    IP_list = []
    i = 0
    while i < size:
        network_address_int = IP_str_to_int(network_address)
        network_address_int += i
        IP_list.append(IP_int_to_str(network_address_int))
        i += 1
        
    return IP_list

def simple_df(df):
    # Takes a dataframe of IP addresses and returns a dataframe with just the reserved addresses and ping replies
    short_df = pd.DataFrame(columns = ['IP address', 'reserved address', 'ICMP reply', 'hostname'])
    
    i = 0
    for index, row in df.iterrows():
        if row['reserved address'] != '' or row['ICMP reply']:
            short_df.loc[i] = [row[0], row[1], row[2], row[3]]
            i += 1
            
    return short_df

## LAN Sweep Function

In [ ]:
from icmplib import multiping
import pandas as pd

def LAN_sweep():
    # Will send ICMP requests to every IP address in the LAN of the host. Will summarize the results in a Pandas dataframe.
    network_address = get_network_address()
    subnet_mask = get_subnet_mask()
    default_gateway = get_default_gateway()
    host_IP = get_host_IP_address()
    broadcast_address = get_broadcast_address()
    LAN = get_LAN()
    IPdf = pd.DataFrame(columns = ['IP address', 'reserved address', 'ICMP reply', 'hostname'])
    
    IP_list = network_IP_list(network_address, subnet_mask)
    
    hosts = multiping(IP_list)
    
    i = 0
    for host in hosts:
        # Assign reserved addresses from the IP address
        reserved_name = ''
        hostname = ''
        
        if host.address == network_address:
            reserved_name = 'network address'
        elif host.address == default_gateway:
            reserved_name = 'default gateway'
        elif host.address == host_IP:
            reserved_name = 'local'
            hostname = get_hostname()
        elif host.address == broadcast_address:
            reserved_name = 'broadcast address'
            
        # Get the hostname
        if host.is_alive and reserved_name == '':
            hostname = get_hostname(host.address)
            
        IPdf.loc[i] = [host.address, reserved_name, host.is_alive, hostname]
        i += 1
                
    return IPdf

## Executable

In [ ]:
IPdf = LAN_sweep()

In [ ]:
df = simple_df(IPdf)

df.style.set_caption(get_LAN()).set_table_styles([{
    'selector': 'caption',
    'props': [
        ('font-weight', 'bold'),
        ('font-size', '12pt')
    ]
}])